# SiNE Scene Viewer

Interactive 3D viewer for Mitsuba XML scene files.

**Controls:**
- **Mouse left**: Rotate
- **Scroll wheel**: Zoom
- **Mouse right**: Pan
- **Alt + click**: Pick point coordinates

In [1]:
from sionna.rt import load_scene
import drjit as dr

# Change this to view different scenes
SCENE_FILE = "two_room_default.xml"

# Use merge_shapes=False to keep individual surfaces separate (for inspection)
scene = load_scene(SCENE_FILE, merge_shapes=False)

# List all scene objects (surfaces) with their IDs, positions, and sizes
print("Scene Objects (surfaces):")
print("=" * 80)
print(f"{'ID':<25} {'Center (x,y,z)':<25} {'Size (w,h,d)':<25}")
print("-" * 80)

for name, obj in scene.objects.items():
    # Get center position (axis-aligned bounding box center)
    # Use drjit to extract scalar values
    pos = obj.position
    px, py, pz = pos[0][0], pos[1][0], pos[2][0]
    center = f"({px:.2f}, {py:.2f}, {pz:.2f})"
    
    # Get bounding box from Mitsuba mesh to calculate size
    bbox = obj.mi_mesh.bbox()
    size_x = bbox.max[0] - bbox.min[0]
    size_y = bbox.max[1] - bbox.min[1]
    size_z = bbox.max[2] - bbox.min[2]
    size = f"({size_x:.2f}, {size_y:.2f}, {size_z:.2f})"
    
    print(f"{name:<25} {center:<25} {size:<25}")

print("-" * 80)
print(f"Total: {len(scene.objects)} objects")
print("\nTip: Use Alt+click in preview to get coordinates of any point")

scene.preview()

Scene Objects (surfaces):
ID                        Center (x,y,z)            Size (w,h,d)             
--------------------------------------------------------------------------------
floor                     (5.07, 2.00, 0.00)        (20.30, 8.00, 0.00)      
ceiling                   (5.07, 2.00, 2.50)        (20.30, 8.00, 0.00)      
room1_wall_back           (2.50, 0.00, 1.25)        (10.00, 0.00, 5.00)      
room1_wall_front          (2.50, 4.00, 1.25)        (10.00, 0.00, 5.00)      
room1_wall_left           (0.00, 2.00, 1.25)        (0.00, 5.00, 8.00)       
divider_above_door        (5.00, 2.00, 2.25)        (0.00, 1.00, 8.00)       
divider_left              (5.00, 0.78, 1.25)        (0.00, 5.00, 3.10)       
divider_right             (5.00, 3.22, 1.25)        (0.00, 5.00, 3.10)       
room2_wall_back           (7.57, 0.00, 1.25)        (10.00, 0.00, 5.00)      
room2_wall_front          (7.57, 4.00, 1.25)        (10.00, 0.00, 5.00)      
room2_wall_right          (10.15, 2

## View with Axis Markers

Add TX/RX devices at origin and along axes as visual markers:

In [2]:
from sionna.rt import Transmitter, Receiver, PlanarArray

# Reload scene to add axis markers
scene_with_axes = load_scene(SCENE_FILE, merge_shapes=False)

# Configure minimal antenna arrays
scene_with_axes.tx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5,
                                        horizontal_spacing=0.5, pattern="iso", polarization="V")
scene_with_axes.rx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5,
                                        horizontal_spacing=0.5, pattern="iso", polarization="V")

# Add markers: Origin (green=TX), and axis endpoints (blue=RX)
# Origin marker
scene_with_axes.add(Transmitter(name="origin", position=[0, 0, 0]))

# X-axis marker (1m along X)
scene_with_axes.add(Receiver(name="x_axis_1m", position=[1, 0, 0]))

# Y-axis marker (1m along Y) 
scene_with_axes.add(Receiver(name="y_axis_1m", position=[0, 1, 0]))

# Z-axis marker (1m along Z)
scene_with_axes.add(Receiver(name="z_axis_1m", position=[0, 0, 1]))

print("Axis markers added:")
print("  Green (TX) = Origin (0,0,0)")
print("  Blue (RX)  = X-axis (1,0,0), Y-axis (0,1,0), Z-axis (0,0,1)")

scene_with_axes.preview()

Axis markers added:
  Green (TX) = Origin (0,0,0)
  Blue (RX)  = X-axis (1,0,0), Y-axis (0,1,0), Z-axis (0,0,1)


## View with Clipping

Use `clip_at` to cut away ceiling/walls and see inside:

In [3]:
# Preview with ceiling clipped away (z=2.0 cuts below 2.5m ceiling)
scene.preview(clip_at=2.0)

## Add Devices and View Paths

Add transmitters/receivers and visualize propagation paths:

In [4]:
from sionna.rt import Transmitter, Receiver, PlanarArray, PathSolver

# Reload scene fresh (merge_shapes=False to keep individual surfaces)
scene = load_scene(SCENE_FILE, merge_shapes=False)

# Configure antenna arrays
scene.tx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5,
                              horizontal_spacing=0.5, pattern="iso", polarization="V")
scene.rx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5,
                              horizontal_spacing=0.5, pattern="iso", polarization="V")

# Add devices (adjust positions for your scene)
tx = Transmitter(name="tx", position=[2.5, 2.0, 1.5])
rx = Receiver(name="rx", position=[7.5, 2.0, 1.0])
scene.add(tx)
scene.add(rx)

# Compute paths
solver = PathSolver()
paths = solver(scene)

# Preview with paths
scene.preview(paths=paths, clip_at=2.0)

## Scene Object Details

List all objects with their materials:

In [5]:
# Detailed object information with bounding box corners
print("Detailed Object Geometry:")
print("=" * 90)

for name, obj in scene.objects.items():
    bbox = obj.mi_mesh.bbox()
    material = obj.radio_material.name if obj.radio_material else "None"
    # Extract scalar values from DrJit types
    pos = obj.position
    px, py, pz = pos[0][0], pos[1][0], pos[2][0]
    min_x, min_y, min_z = bbox.min[0], bbox.min[1], bbox.min[2]
    max_x, max_y, max_z = bbox.max[0], bbox.max[1], bbox.max[2]
    
    print(f"\n{name}:")
    print(f"  Material: {material}")
    print(f"  Center:   ({px:.2f}, {py:.2f}, {pz:.2f})")
    print(f"  Min:      ({min_x:.2f}, {min_y:.2f}, {min_z:.2f})")
    print(f"  Max:      ({max_x:.2f}, {max_y:.2f}, {max_z:.2f})")
    print(f"  Size:     ({max_x - min_x:.2f} x {max_y - min_y:.2f} x {max_z - min_z:.2f})")

Detailed Object Geometry:

floor:
  Material: itu_concrete
  Center:   (5.07, 2.00, 0.00)
  Min:      (-5.07, -2.00, 0.00)
  Max:      (15.22, 6.00, 0.00)
  Size:     (20.30 x 8.00 x 0.00)

ceiling:
  Material: itu_concrete
  Center:   (5.07, 2.00, 2.50)
  Min:      (-5.07, -2.00, 2.50)
  Max:      (15.22, 6.00, 2.50)
  Size:     (20.30 x 8.00 x 0.00)

room1_wall_back:
  Material: itu_concrete
  Center:   (2.50, 0.00, 1.25)
  Min:      (-2.50, -0.00, -1.25)
  Max:      (7.50, 0.00, 3.75)
  Size:     (10.00 x 0.00 x 5.00)

room1_wall_front:
  Material: itu_concrete
  Center:   (2.50, 4.00, 1.25)
  Min:      (-2.50, 4.00, -1.25)
  Max:      (7.50, 4.00, 3.75)
  Size:     (10.00 x 0.00 x 5.00)

room1_wall_left:
  Material: itu_concrete
  Center:   (0.00, 2.00, 1.25)
  Min:      (-0.00, -0.50, -2.75)
  Max:      (0.00, 4.50, 5.25)
  Size:     (0.00 x 5.00 x 8.00)

divider_above_door:
  Material: itu_concrete
  Center:   (5.00, 2.00, 2.25)
  Min:      (5.00, 1.50, -1.75)
  Max:      (5.00, 

## Available Scenes

Change `SCENE_FILE` above to one of:
- `two_room_default.xml` - Standard two-room layout (5m x 4m rooms)
- `two_room_large.xml` - Larger two-room layout (10m x 8m rooms)